In [128]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import json
!pip install -U sentence-transformers
from transformers import AutoTokenizer, AutoModel
from tqdm.auto import tqdm
import timeit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [129]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [104]:
import os
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [105]:
# Allowed to make changes.

# Pre-processing cell. You can use this cell to pre-process input data or load
# your models.
# Allowed to make changes.

# Pre-processing cell. You can use this cell to pre-process input data or load
# your models.
device = "cuda" if torch.cuda.is_available() else "cpu"
input_paragraph_file = "/content/inputs/paragraphs.csv" #"input_paragraph.csv"
theme_interval_file = "/content/drive/MyDrive/devrev/sample_theme_interval_1.csv" #"theme_interval.csv"
input_question_file = "/content/drive/MyDrive/devrev/sample_input_question_1.csv" #"input_question.csv"
output_file = "output.csv" #"output.csv"
"""dict_theme_ind[<THEME>]['<start|end>']"""
df_paras = pd.read_csv(input_paragraph_file, index_col=0)
para_startInd = pd.read_csv(theme_interval_file)
questions = pd.read_csv(input_question_file)

# theme_para_dict = {}
# dict_theme_ind = {}
# dict_paras = df_paras.to_dict()
# for i in range(1,len(dict_paras['theme'].keys())+1):
#   if dict_paras['theme'][i] in theme_para_dict:
#     theme_para_dict[dict_paras['theme'][i]].append(dict_paras['paragraph'][i])
#   else:
#     theme_para_dict[dict_paras['theme'][i]] = [(dict_paras['paragraph'][i])] 
#     dict_theme_ind[dict_paras['theme'][i]] = []

# Dummy code.
# paragraph_ds = []
# global_model = {}

In [106]:
print(len(df_paras),len(para_startInd),len(questions))

1179 30 3917


In [107]:
for x in list(questions["theme"].unique()):
    if ('<' in x) or ('>' in x):
        print(x)

In [108]:
themes = questions['theme'].unique()
questions = questions[questions["theme"].isin(themes)]
df_paras = df_paras[df_paras['theme'].isin(themes)]

In [109]:
print(len(questions),len(df_paras))

3917 1179


In [110]:
# questions['theme'] = ['<' + x.lower() + '>' for x in questions["theme"].values]
# df_paras['theme'] = ['<' + x.lower() + '>' for x in df_paras["theme"].values]

In [111]:
questions['theme'].unique()

array(['IPod', '2008_Sichuan_earthquake', 'Wayback_Machine',
       'Canadian_Armed_Forces', 'Cardinal_(Catholicism)',
       'Human_Development_Index', 'Heresy', 'Warsaw_Pact', 'Materialism',
       'Pub', 'Web_browser', 'Catalan_language', 'Paper',
       'Adult_contemporary_music', 'Nanjing', 'Dialect', 'Southampton',
       'The_Times', 'Immunology', 'Imamah_(Shia_doctrine)', 'Grape',
       'United_States_dollar', 'Everton_F.C.', 'Hard_rock',
       'Great_Plains', 'Biodiversity', 'Federal_Bureau_of_Investigation',
       'Mary_(mother_of_Jesus)', 'Unknown', 'DevRev'], dtype=object)

In [112]:
df = pd.DataFrame.copy(questions)
df.head()

,id,question,theme
0,1,Which company produces the iPod?,IPod
1,2,When was the original iPod released?,IPod
2,3,How many different types of iPod are currently...,IPod
3,4,What kind of device is the iPod?,IPod
4,5,The iPod Touch uses what kind of interface?,IPod


In [113]:
df.isnull().sum()

id          0
question    0
theme       0
dtype: int64

In [114]:
df['answers'] = ""
df.drop('theme',axis = 1, inplace = True)
df.head()

,id,question,answers
0,1,Which company produces the iPod?,
1,2,When was the original iPod released?,
2,3,How many different types of iPod are currently...,
3,4,What kind of device is the iPod?,
4,5,The iPod Touch uses what kind of interface?,


In [115]:
from sklearn.model_selection import train_test_split
# theme_list = list(df['theme'].unique())
# exclude_theme = random.sample(theme_list,3)
# print(exclude_theme)
# remaining_themes = [x for x in theme_list if x not in exclude_theme]
# print(len(remaining_themes))
# train_df = df[df['theme'].isin(remaining_themes)]
# test_df = df[df['theme'].isin(exclude_theme)]

In [116]:
test_data = json.loads(df.to_json(orient="records"))

In [117]:
# def makedict(data):
#     for i in range(len(data)):
#         data[i]['positive_ctxs'] = [{'title':"",'text':data[i]['positive_ctxs']}]
#         data[i]['negative_ctxs'] = [{'title':"",'text':data[i]['negative_ctxs']}]
#         data[i]['hard_negative_ctxs'] = []
#     return data

In [118]:
# train_data = makedict(train_data)
# test_data = makedict(test_data)

In [119]:
with open("test.csv",'w') as fp:
    json.dump(test_data,fp)

In [120]:
!rm -rf DPR

In [121]:
!git clone https://github.com/chidaksh/DPR.git
!cd DPR; pip install .;

Cloning into 'DPR'...
remote: Enumerating objects: 1238, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 1238 (delta 16), reused 22 (delta 9), pack-reused 1205
Receiving objects: 100% (1238/1238), 489.31 KiB | 5.10 MiB/s, done.
Resolving deltas: 100% (832/832), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/DPR
  Preparing metadata (setup.py) ... done
  Created wheel for dpr: filename=dpr-1.0.0-py3-none-any.whl size=14216 sha256=66ac8081e8646ffece7abefa5dc865db7014edfb5b6102cd94aec5af830dbbb0
  Stored in directory: /tmp/pip-ephem-wheel-cache-8e16pyt6/wheels/f5/7e/d3/61296655e7857852a5cc70e05fd5c2c111599dee6b3b2a8424
Successfully built dpr
  Attempting uninstall: dpr
    Found existing installation: dpr 1.0.0
    Uninstalling dpr-1.0.0:
      Successfully uninstalled dpr-1.0.0


In [122]:
# !pwd
# !cd DPR; python train_dense_encoder.py \
# train_datasets=nq_train \
# dev_datasets=nq_dev \
# train=biencoder_local \
# output_dir="../results"
# # ../../../../train.csv
# # ../../../../test.csv

In [131]:
para_df = pd.DataFrame()
para_df["paragraph"] = df_paras["paragraph"]
para_df['title'] = ""
para_df.head()
para_df.to_csv("para.csv")

In [132]:
# !cd DPR; python generate_dense_embeddings.py \
# 	model_file="../results/dpr_biencoder.8" \
# 	ctx_src=dpr_wiki \
# 	out_file="../results/embeddings"	

In [133]:
!pwd

/content


In [136]:
!cd DPR; python dense_retriever.py \
	model_file="../../../../drive/MyDrive/devrev/dpr_biencoder.4"\
	qa_dataset=nq_test \
	ctx_datatsets=[dpr_wiki] \
	encoded_ctx_files=["../../../../drive/MyDrive/devrev/paragraph_embeddings"] \
	out_file="../../../../drive/MyDrive/devrev/retrieve_passages"

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-02-05 12:54:47.816355: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-02-05 12:54:49,042 [INFO] faiss.loader: Loading faiss with AVX2 support.
2023-02-05 12:54:49,059 [INFO] faiss.loader: Successfully loaded faiss with AVX2 support.
dense_retriever.py:474: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  @hydra.main(config_path="conf", config_name="dense_retriever")
/usr/local/lib/python3.8/dist-packages/hydra/_internal/defaults_list.py:251: UserWarning: In 'dense_retriever': Defaults list is missing `_self_`. See https://hydra.cc/docs/1.2/upgrades/1.0_to_1.1/default_composition_order for more information
  warnings.warn(msg, UserWarning)
/usr/loca

In [94]:
import csv
with open("test.csv") as ifile:
    reader = csv.reader(ifile, delimiter=",")
    for row in reader:
        print(row)
        print(type(row))
        question = row[0]
        answers = eval(row[1])
        id = None

['[{"id": 1', ' "question": "Which company produces the iPod?"', ' "answers": ""}', ' {"id": 2', ' "question": "When was the original iPod released?"', ' "answers": ""}', ' {"id": 3', ' "question": "How many different types of iPod are currently available?"', ' "answers": ""}', ' {"id": 4', ' "question": "What kind of device is the iPod?"', ' "answers": ""}', ' {"id": 5', ' "question": "The iPod Touch uses what kind of interface?"', ' "answers": ""}', ' {"id": 6', ' "question": "What company manufactures the iPod?"', ' "answers": ""}', ' {"id": 7', ' "question": "In addition to playing music', ' what other function can the iPod perform?"', ' "answers": ""}', ' {"id": 8', ' "question": "What is the smallest data capacity for an iPod product?"', ' "answers": ""}', ' {"id": 9', ' "question": "What is the largest data capacity for an iPod product?"', ' "answers": ""}', ' {"id": 10', ' "question": "Which iPod product features the smallest data storage capacity?"', ' "answers": ""}', ' {"id"

SyntaxError: ignored

In [140]:
import re
new_df = pd.DataFrame()
questions = []
# answers = []
para = []
hasAnswer = []
ID = []
with open("/content/drive/MyDrive/devrev/retrieve_passages") as f:
    jobj = json.load(f)
    for jline in jobj:
        s = jline["question"]
        questions.append(re.sub('<[^>]+>', '', s))
        # answers.append(jline['answers'])
        para.append(jline['ctxs'][0]['text'])
        hasAnswer.append(jline['ctxs'][0]['has_answer'])
        ID.append(jline['ctxs'][0]['id'])

In [141]:
new_df['question'] = questions
# new_df['answer'] = answers
new_df['para'] = para
new_df['has_answer'] = hasAnswer
new_df['id'] = ID

In [142]:
new_df.head()

,question,para,has_answer,id
0,Which company produces the iPod?,Many accessories have been made for the iPod l...,False,19
1,When was the original iPod released?,The iPod is a line of portable media players a...,False,1
2,How many different types of iPod are currently...,"Like other digital music players, iPods can se...",False,2
3,What kind of device is the iPod?,iPods with color displays use anti-aliased gra...,False,25
4,The iPod Touch uses what kind of interface?,iPods with color displays use anti-aliased gra...,False,25


In [143]:
len(new_df[new_df['has_answer'] == False])/len(new_df)

1.0

In [144]:
new_df.to_csv("./DataForQAModel.csv")

In [37]:
import string
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords

def preprocess_text(text):
    # Tokenize the text into words
    tokens = nltk.word_tokenize(text)

    # Lowercase the text
    tokens = [word.lower() for word in tokens]

    # Remove stop words, punctuation, and any other irrelevant characters
    stop_words = set(stopwords.words("english"))
    tokens = [word for word in tokens if word not in stop_words and word not in string.punctuation]

    return tokens

# Example usage: Pre-process a sample input question
question = "What is the capital of France?"
question_tokens = preprocess_text(question)
print(question_tokens)

# Example usage: Pre-process a sample input passage
passage = "Paris is the capital of France. It is located in the northern central part of the country."
passage_tokens = preprocess_text(passage)
print(passage_tokens)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


['capital', 'france']
['paris', 'capital', 'france', 'located', 'northern', 'central', 'part', 'country']


In [34]:
def get_theme_model(theme):
  # Here, you can load and return a model which is fine tuned for a theme.
  # In case, you only have a global model, you can return that.
  return model

In [ ]:
class QAModel(nn.Module):
    # to start with Hin can be taken as L which is same as number of input tokens
    def __init__(self,pe,qe,inputdim,hiddendim,outputdim): 
        super(QAModel, self).__init__()
        self.PassageEmbeddings = pe
        self.QuestionEmbeddings = qe
        self.inputdim = inputdim
        self.hiddendim = hiddendim
        self.outputdim = outputdim
        self.a = None
        self.bilstm = nn.LSTM(self.inputdim,self.hiddendim,batch_first = True,bidirectional = True) # (N,L,Hin) ---> (N,L,2*Hin) if proj_size = 0
        self.linearLayer = nn.Linear(self.lastLayer,self.outputdim)
        self.maxpool = nn.MaxPool2d((self.a,1), stride=(self.a,1))
        self.avgpool = nn.AvgPool2d((self.a,1), stride=(self.a,1))
        self.softmax = nn.Softmax(dim = 1)

    def forward(self):
        x = self.bilstm(self.PassageEmbeddings)                   # (N,L1,2*Hin)
        y = self.bilstm(self.QuestionEmbeddings)                  # (N,L2,2*Hin)
        z = x @ y.transpose(0,2,1)            # (N,L1,L2)
        self.a = z.shape[1]
        z1 = self.maxpool(z.unsqueeze(1)).squeeze(1)           # (N,1,L1,L2) ----> (N,1,1,L2) ---> (N,1,L2)
        z2 = self.avgpool(z.unsqueeze(1)).squeeze(1)           # (N,1,L1,L2) ----> (N,1,1,L2) ---> (N,1,L2)
        z_concat = torch.cat((z1,z2),dim = 0)               # (N,1,2*L2)
        self.inputdim ,self.hiddendim = z_concat.shape[1],z_concat.shape[2]
        lastLayer = self.bilstm(z_concat)            # (N,1,2*L2) ---> (N,1,4*L2)
        lastLayer = lastLayer.squeeze(1) # (N,4*L2)
        self.lastLayer = lastLayer.shape[1]
        lastLayer = self.linearLayer(lastLayer)
        output = self.softmax(lastLayer) # (N,4*L2) ---> (N,4*L2) where dim = 1 is a probability distribution

In [ ]:
model = QAModel().to(device)

In [ ]:
loss_fn = nn.CrossEntropyLoss()
opt = optim.Adam(model.parameters(), lr = 5*1e-5, weight_decay = 0.01)

# DPR